In [1]:
import pandas as pd
import numpy as np
from cnmf import cNMF, load_df_from_npz
import scanpy as sc
import os
import scipy.sparse as sp
import yaml
import requests

In [2]:
test_dir = '/data/srlab1/TCAT/Data/starCAT/Testing/'
code_dir = '../../Code/starCAT/'

In [3]:
import sys
sys.path.append(code_dir)
from starcat import starCAT

## Test initialization

In [4]:
cat_obj = starCAT()

Using reference from starCAT database
Loading reference from existing cache file for reference TCAT.V1


In [5]:
cat_obj.ref.iloc[0:5, 0:5]

,A1BG,AARD,AARSD1,ABCA1,ABCB1
CellCycle-G2M,2.032614,22.965553,17.423538,3.478179,2.297279
Translation,35.445282,0.000000,9.245893,0.477994,0.000000
HLA,18.192997,14.632670,2.686475,3.937182,0.000000
ISG,0.436212,0.000000,18.078197,17.354506,0.000000
Mito,10.293049,0.000000,52.669895,14.615502,3.341488


In [12]:
cat_obj.scores, cat_obj.score_data

(None,
 {'scores': {'continuous': [{'name': 'ASA',
     'normalization': 'normalized',
     'columns': ['TIMD4/TIM3', 'ICOS/CD38', 'CTLA4/CD38', 'OX40/EBI3']},
    {'name': 'Proliferation',
     'normalization': 'normalized',
     'columns': ['CellCycle-G2M', 'CellCycle-S', 'CellCycle-Late-S']}],
   'discrete': [{'name': 'ASA_binary',
     'normalization': 'normalized',
     'columns': ['TIMD4/TIM3', 'ICOS/CD38', 'CTLA4/CD38', 'OX40/EBI3'],
     'threshold': 0.0625},
    {'name': 'Proliferation_binary',
     'normalization': 'normalized',
     'columns': ['CellCycle-G2M', 'CellCycle-S', 'CellCycle-Late-S'],
     'threshold': 0.1},
    {'name': 'Multinomial_Label',
     'normalization': 'normalized',
     'file': 'multinomial_lineage_classifier.py',
     'function': 'compute_lineage'}]}})

In [6]:
cat_obj.ref_name, cat_obj.score_path

('TCAT.V1', './cache/TCAT.V1/TCAT.V1.scores.yaml')

In [7]:
cat_obj = starCAT(reference = './cache/TCAT.V1/TCAT.V1.reference.tsv')


Using user specified reference spectra file ./cache/TCAT.V1/TCAT.V1.reference.tsv
No scores provided


In [8]:
cat_obj.ref.iloc[0:5, 0:5]

,A1BG,AARD,AARSD1,ABCA1,ABCB1
CellCycle-G2M,2.032614,22.965553,17.423538,3.478179,2.297279
Translation,35.445282,0.000000,9.245893,0.477994,0.000000
HLA,18.192997,14.632670,2.686475,3.937182,0.000000
ISG,0.436212,0.000000,18.078197,17.354506,0.000000
Mito,10.293049,0.000000,52.669895,14.615502,3.341488


In [10]:
cat_obj.scores, cat_obj.score_data

(None, {})

In [13]:
reference = 'other'
cat_obj = starCAT(reference = reference)

Using reference from starCAT database


Exception: other is not found in list of pre-built reference names. It is also not a valid path to a reference file which would need to end in .tsv or .txt. Please provide a valid file path or a reference string from among the following [TCAT.V1]

## Test ```load_counts```

In [5]:
counts_fn = '/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF.h5ad'
adata = cat_obj.load_counts(counts_fn)

Only considering the two last: ['.20221022FiltForcNMF', '.h5ad'].
Only considering the two last: ['.20221022FiltForcNMF', '.h5ad'].


In [15]:
test_dir

'/data/srlab1/TCAT/Data/starCAT/Testing/'

In [16]:
X = pd.DataFrame(adata.X.todense(), index = adata.obs.index, columns = adata.var.index)
X_filt = X.head(5000)

In [21]:
X_filt

Name_ADT_Fixed,AL627309.1,AL669831.5,LINC00115,FAM41C,NOC2L,KLHL17,PLEKHN1,AL645608.8,HES4,ISG15,...,AB_CD169,AB_CD28,AB_CD161,AB_CD163,AB_CD138-1,AB_CD164,AB_CD138-2,AB_CD144,AB_CD202b,AB_CD11c
L1_AAACCCAAGACATACA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,61.0,12.0,8.0,4.0,1.0,3.0,7.0,10.0,22.0
L1_AAACCCACAACTGGTT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,37.0,0.0,6.0,5.0,1.0,13.0,7.0,8.0,13.0
L1_AAACCCACAGCATACT,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,46.0,0.0,1.0,4.0,1.0,12.0,5.0,13.0,12.0
L1_AAACCCACATCAGTCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,67.0,2.0,6.0,6.0,2.0,7.0,7.0,16.0,8.0
L1_AAACCCATCCACACCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,37.0,1.0,7.0,4.0,2.0,11.0,8.0,13.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L1_GTAGGTTAGTGGGAAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,56.0,1.0,4.0,6.0,3.0,9.0,7.0,7.0,11.0
L1_GTAGGTTCAATGAGCG,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,39.0,1.0,6.0,6.0,0.0,15.0,7.0,7.0,16.0
L1_GTAGGTTGTAGGAGGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,45.0,0.0,1.0,5.0,1.0,5.0,7.0,5.0,17.0
L1_GTAGGTTGTTTACACG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,63.0,3.0,7.0,1.0,3.0,17.0,7.0,14.0,19.0


In [16]:
fname = 'haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF'

In [22]:
# X_filt.to_csv(os.path.join(test_dir, '%s.txt' % fname), sep = '\t')
# save_df_to_npz(X_filt, os.path.join(test_dir, '%s.npz' % fname))

In [ ]:
counts_fn = os.path.join(test_dir, '%s.txt' % fname)
adata = cat_obj.load_counts(counts_fn)

In [21]:
adata, adata.X

(AnnData object with n_obs × n_vars = 5000 × 20957,
 <5000x20957 sparse matrix of type '<class 'numpy.float32'>'
 	with 8102998 stored elements in Compressed Sparse Row format>)

In [22]:
display(adata.obs.head(2)), display(adata.var.head(2))

""
L1_AAACCCAAGACATACA
L1_AAACCCACAACTGGTT


""
AL627309.1
AL669831.5


(None, None)

In [14]:
# sc.read_10x_mtx only supports direct cellranger outputs (i.e. compressed)
# Load from sample 10X matrix 
counts_fn = os.path.join(test_dir, '500_PBMC_3p_LT_Chromium_Controller/raw_feature_bc_matrix/matrix.mtx.gz')
adata = cat_obj.load_counts(counts_fn)

In [15]:
adata, adata.X

(AnnData object with n_obs × n_vars = 13785 × 36601
     var: 'gene_ids', 'feature_types',
 <13785x36601 sparse matrix of type '<class 'numpy.float32'>'
 	with 2826293 stored elements in Compressed Sparse Row format>)

In [12]:
display(adata.obs.head(2)), display(adata.var.head(2))

""
AATCACGAGAAATTGC-1
AATCACGAGAAGAAAC-1


,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression


(None, None)

## Test ```fit_transform```

In [17]:
counts_fn = '/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF.h5ad'
adata = cat_obj.load_counts(counts_fn)

Only considering the two last: ['.20221022FiltForcNMF', '.h5ad'].
Only considering the two last: ['.20221022FiltForcNMF', '.h5ad'].


In [18]:
usage, scores = cat_obj.fit_transform(adata)

3412 out of 3412 genes in the reference overlap with the query


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [20]:
usage.head(2)

,CellCycle-G2M,Translation,HLA,ISG,Mito,Doublet-RBC,gdT,CellCycle-S,Cytotoxic,Doublet-Platelet,...,Tfh-2,OX40/EBI3,CD172a/MERTK,IEG3,Doublet-Fibroblast,SOX4/TOX2,CD40LG/TXNIP,Tph,Exhaustion,Tfh-1
L1_AAACCCAAGACATACA,0.001557,0.171755,0.002625,0.001945,0.005562,0.001173,0.003777,0.005649,0.003786,0.002121,...,0.010761,0.001597,0.090117,0.030178,0.002268,0.007337,0.072857,0.001429,0.002088,0.008598
L1_AAACCCACAACTGGTT,0.000774,0.214818,0.005179,0.003329,0.031739,0.001086,0.004772,0.006168,0.007771,0.005516,...,0.004503,0.001213,0.125031,0.022496,0.008215,0.009476,0.007855,0.000929,0.001617,0.002818


In [22]:
scores.head(2)

,ASA,Proliferation,ASA_binary,Proliferation_binary,Multinomial_Label
L1_AAACCCAAGACATACA,0.024724,0.009160,False,False,CD4_CM
L1_AAACCCACAACTGGTT,0.028785,0.007687,False,False,CD8_Naive


In [23]:
adata.X = adata.X.todense()

In [11]:
usage, scores = cat_obj.fit_transform(adata)

3412 out of 3412 genes in the reference overlap with the query


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [25]:
usage.head(2)

,CellCycle-G2M,Translation,HLA,ISG,Mito,Doublet-RBC,gdT,CellCycle-S,Cytotoxic,Doublet-Platelet,...,Tfh-2,OX40/EBI3,CD172a/MERTK,IEG3,Doublet-Fibroblast,SOX4/TOX2,CD40LG/TXNIP,Tph,Exhaustion,Tfh-1
L1_AAACCCAAGACATACA,0.001557,0.171756,0.002625,0.001945,0.005562,0.001173,0.003777,0.005649,0.003786,0.002121,...,0.010761,0.001597,0.090116,0.030178,0.002268,0.007337,0.072858,0.001429,0.002088,0.008598
L1_AAACCCACAACTGGTT,0.000774,0.214819,0.005179,0.003329,0.031739,0.001086,0.004772,0.006168,0.007772,0.005516,...,0.004503,0.001213,0.125030,0.022496,0.008215,0.009476,0.007855,0.000929,0.001617,0.002818


In [26]:
scores.head(2)

,ASA,Proliferation,ASA_binary,Proliferation_binary,Multinomial_Label
L1_AAACCCAAGACATACA,0.024724,0.009160,False,False,CD4_CM
L1_AAACCCACAACTGGTT,0.028785,0.007687,False,False,CD8_Naive


In [28]:
scores['Multinomial_Label'].value_counts()

CD4_Naive    19703
CD4_CM       11039
CD8_Naive    10080
CD4_EM       10052
CD8_EM        6611
CD8_TEMRA     5526
CD8_CM        3727
MAIT          3042
Treg          1976
gdT           1503
Name: Multinomial_Label, dtype: int64

In [29]:
cat_obj_noscore = starCAT(reference = './cache/TCAT.V1/TCAT.V1.reference.tsv')

Using user specified reference spectra file ./cache/TCAT.V1/TCAT.V1.reference.tsv
No scores provided


In [35]:
usage, scores = cat_obj_noscore.fit_transform(adata)

3412 out of 3412 genes in the reference overlap with the query


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [36]:
usage.head(2)

,CellCycle-G2M,Translation,HLA,ISG,Mito,Doublet-RBC,gdT,CellCycle-S,Cytotoxic,Doublet-Platelet,...,Tfh-2,OX40/EBI3,CD172a/MERTK,IEG3,Doublet-Fibroblast,SOX4/TOX2,CD40LG/TXNIP,Tph,Exhaustion,Tfh-1
L1_AAACCCAAGACATACA,0.001557,0.171756,0.002625,0.001945,0.005562,0.001173,0.003777,0.005649,0.003786,0.002121,...,0.010761,0.001597,0.090116,0.030178,0.002268,0.007337,0.072858,0.001429,0.002088,0.008598
L1_AAACCCACAACTGGTT,0.000774,0.214819,0.005179,0.003329,0.031739,0.001086,0.004772,0.006168,0.007772,0.005516,...,0.004503,0.001213,0.125030,0.022496,0.008215,0.009476,0.007855,0.000929,0.001617,0.002818


In [37]:
scores

In [38]:
cat_obj_noscore.score_path

## Test ```save_usage```

In [12]:
test_dir

'/data/srlab1/TCAT/Data/starCAT/Testing/'

In [13]:
fname = 'haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF'

In [14]:
name = 'starCAT_%s' % fname
name

'starCAT_haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF'

In [15]:
output_dir = test_dir
output_dir

'/data/srlab1/TCAT/Data/starCAT/Testing/'

In [ ]:
cat_obj.usage.head(2)

,CellCycle-G2M,Translation,HLA,ISG,Mito,Doublet-RBC,gdT,CellCycle-S,Cytotoxic,Doublet-Platelet,...,Tfh-2,OX40/EBI3,CD172a/MERTK,IEG3,Doublet-Fibroblast,SOX4/TOX2,CD40LG/TXNIP,Tph,Exhaustion,Tfh-1
L1_AAACCCAAGACATACA,0.000015,0.001637,0.000025,0.000019,0.000053,0.000011,0.000036,0.000054,0.000036,0.000020,...,0.000103,0.000015,0.000859,0.000288,0.000022,0.000070,0.000695,0.000014,0.000020,0.000082
L1_AAACCCACAACTGGTT,0.000006,0.001617,0.000039,0.000025,0.000239,0.000008,0.000036,0.000046,0.000058,0.000042,...,0.000034,0.000009,0.000941,0.000169,0.000062,0.000071,0.000059,0.000007,0.000012,0.000021


In [18]:
cat_obj.scores.head(2)

,ASA,Proliferation,ASA_binary,Proliferation_binary,Multinomial_Label
L1_AAACCCAAGACATACA,0.024724,0.009160,False,False,CD4_CM
L1_AAACCCACAACTGGTT,0.028785,0.007687,False,False,CD8_Naive


In [19]:
cat_obj.save_results(output_dir, name)

Saving usages to /data/srlab1/TCAT/Data/starCAT/Testing/starCAT_haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF.rf_usage_normalized.txt
Saving scores to /data/srlab1/TCAT/Data/starCAT/Testing/starCAT_haoetal_pbmc_multimodal.merged.T.raw.ADTfixedADT_70.20221022FiltForcNMF.scores.txt


## Test ```build_reference```

In [20]:
ref_dir = '/data/srlab1/mcurtis/GSK/tcell_proliferation/Hao_20220817/cnmf_output/'
ref_name = 'T_learnHarmonyRNA_RefitBoth'

k = 36
density_threshold = 0.2

In [21]:
cat_obj = starCAT()

Using reference from starCAT database
Loading reference from existing cache file for reference TCAT.V1


In [8]:
cat_obj.build_reference(ref_dir, ref_name, k, density_threshold)

Saving reference spectra to /data/srlab1/mcurtis/GSK/tcell_proliferation/Hao_20220817/cnmf_output/T_learnHarmonyRNA_RefitBoth/T_learnHarmonyRNA_RefitBoth.starCAT_reference.k_36.dt_0_2.tsv


In [8]:
# View default reference spectra (genes x programs)
print(cat_obj.ref_name)
display(cat_obj.ref.iloc[:5, :5])

/data/srlab1/mcurtis/GSK/tcell_proliferation/Hao_20220817/cnmf_output/T_learnHarmonyRNA_RefitBoth/T_learnHarmonyRNA_RefitBoth.starCAT_reference.k_36.dt_0_2.tsv


,HES4,ISG15,TNFRSF18,TNFRSF4,MXRA8
GEP1,15.352291,0.0,0.0,0.0,0.000000
GEP2,0.000000,0.0,0.0,0.0,0.000000
GEP3,0.000000,0.0,0.0,0.0,0.000000
GEP4,0.000000,0.0,0.0,0.0,0.000000
GEP5,0.000000,0.0,0.0,0.0,63.397045


starCAT can be immediately run with the new reference spectra. However, after building reference once, the path to the saved reference can be loaded in future initializations of the starCAT object, i.e. 

```cat_obj = starCAT(reference = 'reference_path')```

In [ ]:
data_dir = './Example_Data'
datafn = os.path.join(data_dir, 'example_data.h5ad')

In [15]:
adata = cat_obj.load_counts(datafn)
usage, _ = cat_obj.fit_transform(adata)

1858 out of 2000 genes in the reference overlap with the query


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [12]:
# View default reference spectra (genes x programs)
print(cat_obj.ref_name)
display(cat_obj.ref.iloc[:5, :5])

/data/srlab1/mcurtis/GSK/tcell_proliferation/Hao_20220817/cnmf_output/T_learnHarmonyRNA_RefitBoth/T_learnHarmonyRNA_RefitBoth.starCAT_reference.k_36.dt_0_2.tsv


,HES4,ISG15,TNFRSF18,TNFRSF4,MXRA8
GEP1,15.352291,0.0,0.0,0.0,0.000000
GEP2,0.000000,0.0,0.0,0.0,0.000000
GEP3,0.000000,0.0,0.0,0.0,0.000000
GEP4,0.000000,0.0,0.0,0.0,0.000000
GEP5,0.000000,0.0,0.0,0.0,63.397045


In [16]:
usage.head(2)

,GEP1,GEP2,GEP3,GEP4,GEP5,GEP6,GEP7,GEP8,GEP9,GEP10,...,GEP27,GEP28,GEP29,GEP30,GEP31,GEP32,GEP33,GEP34,GEP35,GEP36
CATGCCTAGTCGATAA-1-gPlexA4,0.030859,0.020843,0.021415,0.014571,0.067726,0.000059,0.004446,0.007970,0.120426,0.004570,...,0.001371,0.000247,0.000027,0.000080,0.000001,0.000014,0.006562,0.000002,0.000089,0.000120
AAGACCTGTAGCGTCC-1-gPlexC6,0.303648,0.173364,0.126317,0.030777,0.006067,0.046658,0.032619,0.004227,0.044378,0.002768,...,0.007880,0.001387,0.000048,0.000873,0.000006,0.000400,0.001437,0.000912,0.000746,0.000035


## End

In [17]:
import session_info
session_info.show()